In [ ]:
from ultralytics import YOLO
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np
import cv2
import os

# --- Load models ---
yolo_model = YOLO('yolov8n.pt')  # Use your trained YOLOv8 model
cnn_model = load_model('astrocyte_live_dead_model.h5')   # Your trained CNN model

# --- Parameters ---
input_image_path = 'test.tif'
confidence_threshold = 0.27
cnn_input_size = (64, 64)  # Resize input for CNN
class_labels = ['Dead', 'Live']

# --- Load image ---
image = cv2.imread(input_image_path)
original = image.copy()

# --- Run YOLOv8 prediction ---
results = yolo_model.predict(source=image, conf=confidence_threshold, save=False)[0]
boxes = results.boxes.data.cpu().numpy() if results.boxes is not None else []

# --- Process each detection ---
for box in boxes:
    x1, y1, x2, y2, conf, cls = box
    if conf < confidence_threshold:
        continue

    x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
    cropped_cell = image[y1:y2, x1:x2]

    # Skip if bounding box is invalid
    if cropped_cell.size == 0:
        continue

    # --- Preprocess for CNN ---
    cell_resized = cv2.resize(cropped_cell, cnn_input_size)
    cell_array = img_to_array(cell_resized) / 255.0
    cell_array = np.expand_dims(cell_array, axis=0)

    # --- Predict with CNN ---
    prediction = cnn_model.predict(cell_array, verbose=0)
    label_idx = np.argmax(prediction)
    label = class_labels[label_idx]

    # --- Draw on image ---
    cv2.rectangle(original, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(original, f'{label} ({conf:.2f})', (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36, 255, 12), 1)

# --- Display and save ---
cv2.imwrite('output_results.jpg', original)
cv2.imshow('Live/Dead Cell Detection', original)
cv2.waitKey(0)
cv2.destroyAllWindows()


C:\Users\19494\anaconda3\envs\tfenv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



0: 416x640 (no detections), 198.1ms
Speed: 4.8ms preprocess, 198.1ms inference, 1.8ms postprocess per image at shape (1, 3, 416, 640)
